In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("vijayaadithyanvg/car-price-predictionused-cars")

print("Path to dataset files:", path)

Path to dataset files: /home/mainuser/.cache/kagglehub/datasets/vijayaadithyanvg/car-price-predictionused-cars/versions/1
